# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import re
import time
import sqlalchemy
from sqlalchemy import create_engine
import io
import psycopg2

from pathlib import Path
from collections import Counter

# Create File Paths and Import As Dataframes

In [2]:
ratings_path = "Starting_Data/ratings.csv"
movies_path = "Active_Data/movies.csv"
genres_path = "Active_Data/movie_genres.csv"
metadata_path = "Starting_Data/export_metadata.csv"
cast_path = "Starting_Data/movie_cast.csv"

In [3]:
ratings_df = pd.read_csv(ratings_path, low_memory=False)
movie_df = pd.read_csv(movies_path, low_memory=False)
genres_df = pd.read_csv(genres_path, low_memory=False)
meta_df = pd.read_csv(metadata_path, low_memory=False)
cast_df = pd.read_csv(cast_path, low_memory=False)

In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [5]:
movie_df.head()

,movie_id,name,year
0,1,Toy Story,1995.0
1,2,Jumanji,1995.0
2,3,Grumpier Old Men,1995.0
3,4,Waiting to Exhale,1995.0
4,5,Father of the Bride Part II,1995.0


In [6]:
genres_df.head()

,mg_id,genre_id,movie_id
0,0,1,1
1,1,3,1
2,2,4,1
3,3,5,1
4,4,9,1


In [7]:
meta_df.head()

,movieId,release_date,runtime
0,1,1995-10-30,81
1,2,1995-12-15,104
2,3,1995-12-22,101
3,4,1995-12-22,127
4,5,1995-02-10,106


In [8]:
cast_df.head()

,movie_id,cast_id
0,1,31
1,2,2157
2,3,6837
3,4,8851
4,5,67773


# Create mc_id Index With Base 1 Not Python's Base 0

In [9]:
cast_df["index"] = cast_df.index
Cast_Index = cast_df["index"] + 1
cast_df["mc_id"] = Cast_Index

In [10]:
meta_df["movie_id"] = meta_df["movieId"]
meta_df = meta_df.drop(columns=["release_date", "movieId"])
meta_df.head()

,runtime,movie_id
0,81,1
1,104,2
2,101,3
3,127,4
4,106,5


# Drop Unnecessary Index Column

In [11]:
cast_df = cast_df.drop(columns=["index"])
cast_df.head()

,movie_id,cast_id,mc_id
0,1,31,1
1,2,2157,2
2,3,6837,3
3,4,8851,4
4,5,67773,5


In [12]:
ratings_df.count()

userId       27753444
movieId      27753444
rating       27753444
timestamp    27753444
dtype: int64

In [13]:
cast_df.count()

movie_id    36790
cast_id     36790
mc_id       36790
dtype: int64

In [14]:
movie_df.count()

movie_id    58098
name        57769
year        57769
dtype: int64

In [15]:
meta_df.count()

runtime     45180
movie_id    45180
dtype: int64

In [16]:
genres_df.count()

mg_id       106104
genre_id    106104
movie_id    106104
dtype: int64

# Drop Duplicates In Dataframes

In [17]:
ratings_df = ratings_df.drop_duplicates(keep="first")
cast_df = cast_df.drop_duplicates(keep="first")
movie_df = movie_df.drop_duplicates(keep="first")
meta_df = meta_df.drop_duplicates(keep="first")
genres_df = genres_df.drop_duplicates(keep="first")

# Drop NaN Values In Dataframes

In [18]:
ratings_df = ratings_df.dropna()
cast_df = cast_df.dropna()
movie_df = movie_df.dropna()
meta_df = meta_df.dropna()
genres_df = genres_df.dropna()

In [19]:
ratings_df.count()

userId       27753444
movieId      27753444
rating       27753444
timestamp    27753444
dtype: int64

In [20]:
cast_df.count()

movie_id    36790
cast_id     36790
mc_id       36790
dtype: int64

In [21]:
movie_df.count()

movie_id    57769
name        57769
year        57769
dtype: int64

In [22]:
meta_df.count()

runtime     45118
movie_id    45118
dtype: int64

In [23]:
genres_df.count()

mg_id       106104
genre_id    106104
movie_id    106104
dtype: int64

# Merge Movie Data With Metadata

In [24]:
movie_df = movie_df.set_index("movie_id")
meta_df = meta_df.set_index("movie_id")

In [25]:
movies_df = movie_df.join(meta_df, on="movie_id", how="left", sort=True)

In [26]:
movies_df.count()

name       57769
year       57769
runtime    45002
dtype: int64

In [27]:
movies_df = movies_df.drop_duplicates(keep="first")
movies_df = movies_df.dropna()

In [28]:
movies_df.count()

name       44994
year       44994
runtime    44994
dtype: int64

# Ensure Column Names and Datatypes Are Consistent With Database

In [29]:
movies_df["year"] = movies_df["year"].astype(int)
movies_df["runtime"] = movies_df["runtime"].astype(int)
movies_df.head()

,name,year,runtime
movie_id,,,
1,Toy Story,1995,81
2,Jumanji,1995,104
3,Grumpier Old Men,1995,101
4,Waiting to Exhale,1995,127
5,Father of the Bride Part II,1995,106


In [30]:
ratings_df["user_id"] = ratings_df["userId"]
ratings_df["movie_id"] = ratings_df["movieId"]
ratings_df.head()

,userId,movieId,rating,timestamp,user_id,movie_id
0,1,307,3.5,1256677221,1,307
1,1,481,3.5,1256677456,1,481
2,1,1091,1.5,1256677471,1,1091
3,1,1257,4.5,1256677460,1,1257
4,1,1449,4.5,1256677264,1,1449


In [31]:
genres_df.head(20)

,mg_id,genre_id,movie_id
0,0,1,1
1,1,3,1
2,2,4,1
3,3,5,1
4,4,9,1
5,5,1,2
6,6,4,2
7,7,9,2
8,8,5,3
9,9,15,3


# Set Dataframe Columns In Same Order As Database And Set Index For Each

In [32]:
genres_df = genres_df.loc[:, ["mg_id", "movie_id", "genre_id"]]
ratings_df = ratings_df.loc[:, ["user_id", "movie_id", "rating"]]
cast_df = cast_df.loc[:, ["mc_id", "movie_id", "cast_id"]]

genres_df = genres_df.set_index("mg_id")
ratings_df = ratings_df.set_index("user_id")
cast_df = cast_df.set_index("mc_id")

In [33]:
genres_df.head()

,movie_id,genre_id
mg_id,,
0,1,1
1,1,3
2,1,4
3,1,5
4,1,9


In [34]:
genres_df.count()

movie_id    106104
genre_id    106104
dtype: int64

In [35]:
movies_df.head()

,name,year,runtime
movie_id,,,
1,Toy Story,1995,81
2,Jumanji,1995,104
3,Grumpier Old Men,1995,101
4,Waiting to Exhale,1995,127
5,Father of the Bride Part II,1995,106


In [36]:
ratings_df.head()

,movie_id,rating
user_id,,
1,307,3.5
1,481,3.5
1,1091,1.5
1,1257,4.5
1,1449,4.5


In [37]:
cast_df.head()

,movie_id,cast_id
mc_id,,
1,1,31
2,2,2157
3,3,6837
4,4,8851
5,5,67773


In [38]:
genres_df.count()

movie_id    106104
genre_id    106104
dtype: int64

In [39]:
movies_df.count()

name       44994
year       44994
runtime    44994
dtype: int64

In [40]:
ratings_df.count()

movie_id    27753444
rating      27753444
dtype: int64

In [41]:
cast_df.count()

movie_id    36790
cast_id     36790
dtype: int64

# Import Sensitive Access Information For Database From Config File

In [42]:
from config import db_password
from config import username

# Establish Connection With AWS Database

In [43]:
host = 'movies-fp.cpige012zhtw.us-east-1.rds.amazonaws.com'
port = 5432
passw = db_password
database = "postgres"
port=5432

   
db_string = "postgresql://" + username + ":" + passw + "@" + host + ":" + "5432/" + database
engine = create_engine(db_string)

# Write Dataframes Into SQL Tables In Database

In [44]:
genres_df.to_sql(name="movie_genre", con=engine, if_exists="replace")

In [ ]:
movies_df.to_sql(name="movies", con=engine, if_exists="replace")

In [ ]:
ratings_df.to_sql(name="ratings", con=engine, if_exists="replace")

In [ ]:
cast_df.to_sql(name="movie_cast", con=engine, if_exists="replace")